In [1]:
# Instalação dos pacotes necessários
!pip install pyspark
!pip install findspark

In [2]:
# Importação e inicialização do findspark

import findspark
findspark.init()


In [35]:
# Importação dos pacotes necessários ao pyspark

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession, functions as F

from pyspark.sql.functions import udf
from pyspark.sql.functions import rank, col

In [4]:
# Criação do contexto do spark
sc = SparkContext()

# Instância do criador de sessão do spark
spark = (SparkSession.builder
                     .master("local[*]")
                     .appName("Etapa de Relatórios - Aceleração PySpark/Capgemini")
                    .getOrCreate())

### Relatório - Qualidade

### Pergunta 1

In [50]:
# Leitura dos arquivos parquet

df_airports_qa = spark.read.parquet("C:/Users/danisant/parquets/airports_qa.parquet")

df_planes_qa = spark.read.parquet("C:/Users/danisant/parquets/planes_qa.parquet")

df_flights_qa = spark.read.parquet("C:/Users/danisant/parquets/flights_qa.parquet")


In [51]:
# verificando quantidade de linhas em cada .parquet


print(df_airports_qa.count())
print(df_planes_qa.count())
print(df_flights_qa.count())


1397
2628
10000


In [52]:
## removendo colunas indesejadas para a análise

df_airports_qa = df_airports_qa.drop('name','lat','lon','alt','tz','dst')
df_airports_qa.printSchema()


df_planes_qa = df_planes_qa.drop('year','type','manufacturer','model','engines','seats','speed','engine')
df_planes_qa.printSchema()


df_flights_qa = df_flights_qa.drop('year','month','day','hour',
                                   'minute','dep_time','arr_time',
                                   'dep_delay','arr_delay','carrier',
                                    'flight','air_time','distance')

df_flights_qa.printSchema()

root
 |-- faa: string (nullable = true)
 |-- qa_faa: string (nullable = true)
 |-- qa_name: string (nullable = true)
 |-- qa_lat: string (nullable = true)
 |-- qa_lon: string (nullable = true)
 |-- qa_alt: string (nullable = true)
 |-- qa_tz: string (nullable = true)
 |-- qa_dst: string (nullable = true)

root
 |-- tailnum: string (nullable = true)
 |-- qa_tailnum: string (nullable = true)
 |-- qa_year: string (nullable = true)
 |-- qa_type: string (nullable = true)
 |-- qa_manufacturer: string (nullable = true)
 |-- qa_model: string (nullable = true)
 |-- qa_engines: string (nullable = true)
 |-- qa_seats: string (nullable = true)
 |-- qa_speed: string (nullable = true)
 |-- qa_engine: string (nullable = true)

root
 |-- tailnum: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- qa_year_month_day: string (nullable = true)
 |-- qa_hour_minute: string (nullable = true)
 |-- qa_dep_arr_time: string (nullable = true)
 |-- qa_dep_arr_de

In [ ]:
## renomeando as colunas para evitar duplicidade

df_planes_qa = (df_planes_qa.withColumnRenamed("tailnum","tailnum_plane")
                            .withColumnRenamed('qa_tailnum', 'qa_tailnum_P'))

In [12]:
# unindo os datasets e criando os relacionamentos entre as colunas necessárias




df = df_flights_qa.join(df_planes_qa,
            df_flights_qa.tailnum == df_planes_qa.tailnum_plane,
            "left")


df = df.join(df_airports_qa,
                        df_flights_qa.origin == df_airports_qa.faa,
                       "left")




df = df.join(df_airports_qa_1,
                        df_flights_qa.dest == df_airports_qa_1.faa_1,
                       "left")


df = df.drop('tailnum','origin','dest','faa','faa_1','tailnum_plane','qa_tailnum_plane')

df.printSchema()

print(df.count())




root
 |-- qa_year_month_day: string (nullable = true)
 |-- qa_hour_minute: string (nullable = true)
 |-- qa_dep_arr_time: string (nullable = true)
 |-- qa_dep_arr_delay: string (nullable = true)
 |-- qa_carrier: string (nullable = true)
 |-- qa_tailnum: string (nullable = true)
 |-- qa_flight: string (nullable = true)
 |-- qa_origin_dest: string (nullable = true)
 |-- qa_air_time: string (nullable = true)
 |-- qa_distance: string (nullable = true)
 |-- qa_distance_airtime: string (nullable = true)
 |-- qa_tailnum_P: string (nullable = true)
 |-- qa_year: string (nullable = true)
 |-- qa_type: string (nullable = true)
 |-- qa_manufacturer: string (nullable = true)
 |-- qa_model: string (nullable = true)
 |-- qa_engines: string (nullable = true)
 |-- qa_seats: string (nullable = true)
 |-- qa_speed: string (nullable = true)
 |-- qa_engine: string (nullable = true)
 |-- qa_faa: string (nullable = true)
 |-- qa_name: string (nullable = true)
 |-- qa_lat: string (nullable = true)
 |-- qa_lo

### Pergunta 2

In [13]:
#
names = df.schema.names

for column in names:
    df = df.withColumn(column, (F.when((F.col(column).startswith("M")) ,"M")
    .when((F.col(column).startswith("F")) ,"F")
    .when((F.col(column).startswith("I")) ,"I")
    .when((F.col(column).startswith("S")) ,"S")
    .when((F.col(column).startswith("T")) ,"T").otherwise(F.col(column))
    ))

    df.filter((F.col(column).isNotNull()) & (F.col(column).isin("M","F","I","S","T"))).select(F.col(column)).groupBy(F.col(column).alias("Issues")).agg(F.count(F.col(column)).cast('int').alias(column)).orderBy(F.col(column)).show()



+------+-----------------+
|Issues|qa_year_month_day|
+------+-----------------+
+------+-----------------+

+------+--------------+
|Issues|qa_hour_minute|
+------+--------------+
|     M|            48|
+------+--------------+

+------+---------------+
|Issues|qa_dep_arr_time|
+------+---------------+
|     M|             55|
|     F|            241|
+------+---------------+

+------+----------------+
|Issues|qa_dep_arr_delay|
+------+----------------+
|     M|              75|
+------+----------------+

+------+----------+
|Issues|qa_carrier|
+------+----------+
+------+----------+

+------+----------+
|Issues|qa_tailnum|
+------+----------+
|     M|        14|
|     F|       989|
+------+----------+

+------+---------+
|Issues|qa_flight|
+------+---------+
|     F|     6158|
+------+---------+

+------+--------------+
|Issues|qa_origin_dest|
+------+--------------+
+------+--------------+

+------+-----------+
|Issues|qa_air_time|
+------+-----------+
|     M|         75|
+------+-

### Pergunta 3


In [34]:
for column in df.columns:
    if 'qa' in column:
        df.filter(F.col(column).isNotNull() & (F.col(column).startswith('M'))).groupBy(F.substring(column, 1, 1).alias(column)).count().show()

+-----------------+-----+
|qa_year_month_day|count|
+-----------------+-----+
+-----------------+-----+

+--------------+-----+
|qa_hour_minute|count|
+--------------+-----+
|             M|   48|
+--------------+-----+

+---------------+-----+
|qa_dep_arr_time|count|
+---------------+-----+
|              M|   55|
+---------------+-----+

+----------------+-----+
|qa_dep_arr_delay|count|
+----------------+-----+
|               M|   75|
+----------------+-----+

+----------+-----+
|qa_carrier|count|
+----------+-----+
+----------+-----+

+----------+-----+
|qa_tailnum|count|
+----------+-----+
|         M|   14|
+----------+-----+

+---------+-----+
|qa_flight|count|
+---------+-----+
+---------+-----+

+--------------+-----+
|qa_origin_dest|count|
+--------------+-----+
+--------------+-----+

+-----------+-----+
|qa_air_time|count|
+-----------+-----+
|          M|   75|
+-----------+-----+

+-----------+-----+
|qa_distance|count|
+-----------+-----+
+-----------+-----+

+----------

### Pergunta 4



In [53]:
for c in df.columns:
    if 'qa' in c:
        df.filter((F.col(c)!='') & (F.col(c).startswith('F'))).groupBy(F.substring(c, 1, 1).alias(c)).count().show()

+-----------------+-----+
|qa_year_month_day|count|
+-----------------+-----+
+-----------------+-----+

+--------------+-----+
|qa_hour_minute|count|
+--------------+-----+
+--------------+-----+

+---------------+-----+
|qa_dep_arr_time|count|
+---------------+-----+
|              F|  241|
+---------------+-----+

+----------------+-----+
|qa_dep_arr_delay|count|
+----------------+-----+
+----------------+-----+

+----------+-----+
|qa_carrier|count|
+----------+-----+
+----------+-----+

+----------+-----+
|qa_tailnum|count|
+----------+-----+
|         F|  989|
+----------+-----+

+---------+-----+
|qa_flight|count|
+---------+-----+
|        F| 6158|
+---------+-----+

+--------------+-----+
|qa_origin_dest|count|
+--------------+-----+
+--------------+-----+

+-----------+-----+
|qa_air_time|count|
+-----------+-----+
+-----------+-----+

+-----------+-----+
|qa_distance|count|
+-----------+-----+
+-----------+-----+

+-------------------+-----+
|qa_distance_airtime|count|
+----

### Pergunta 5



In [54]:
for c in df.columns:
    if 'qa' in c:
        df.filter((F.col(c)!='') & (F.col(c).startswith('I'))).groupBy(F.substring(c, 1, 1).alias(c)).count().show()

+-----------------+-----+
|qa_year_month_day|count|
+-----------------+-----+
+-----------------+-----+

+--------------+-----+
|qa_hour_minute|count|
+--------------+-----+
+--------------+-----+

+---------------+-----+
|qa_dep_arr_time|count|
+---------------+-----+
+---------------+-----+

+----------------+-----+
|qa_dep_arr_delay|count|
+----------------+-----+
+----------------+-----+

+----------+-----+
|qa_carrier|count|
+----------+-----+
+----------+-----+

+----------+-----+
|qa_tailnum|count|
+----------+-----+
+----------+-----+

+---------+-----+
|qa_flight|count|
+---------+-----+
+---------+-----+

+--------------+-----+
|qa_origin_dest|count|
+--------------+-----+
+--------------+-----+

+-----------+-----+
|qa_air_time|count|
+-----------+-----+
+-----------+-----+

+-----------+-----+
|qa_distance|count|
+-----------+-----+
+-----------+-----+

+-------------------+-----+
|qa_distance_airtime|count|
+-------------------+-----+
+-------------------+-----+

+--------

### Relatório - Negócios

In [ ]:
df_airports_bu = (spark.getOrCreate().read
                  .format("parquet")
                  .option("header", "true")
                  .load("C:/Users/danisant/parquets/airports_proc.parquet"))

df_planes_bu = (spark.getOrCreate().read
                  .format("parquet")
                  .option("header", "true")
                  .load("C:/Users/danisant/parquets/planes_proc.parquet"))

df_flights_bu = (spark.getOrCreate().read
                  .format("parquet")
                  .option("header", "true")
                  .load("C:/Users/danisant/parquets/flights_proc.parquet"))

### Pergunta 1

In [ ]:
df_flights_bu = df_flights_bu.select(*(F.col(x).alias('fl_' + x ) for x in df_flights_bu.columns))

df_planes_bu = df_planes_bu.select(*(F.col(x).alias('pl_' + x ) for x in df_planes_bu.columns))


df_airports_bu = df_airports_bu.select(*(F.col(x).alias('air_' + x ) for x in df_airports_bu.columns))

In [ ]:
df_flights_bu.printSchema()
df_planes_bu.printSchema()
df_airports_bu.printSchema()

In [ ]:



df = df_flights_bu.join(df_planes_bu,
            df_flights_bu.fl_tailnum == df_planes_bu.pl_tailnum,
            "left")

df = df.join(df_airports_bu,
                        df_flights_bu.fl_origin == df_airports_bu.air_faa,
                       "left")

df_airports_bu_1 = (df_airports_bu
      
                    .withColumnRenamed("air_faa","air2_faa")
                    .withColumnRenamed("air_alt","air2_alt")
                    .withColumnRenamed("air_lat","air2_lat")
                    .withColumnRenamed("air_lon","air2_lon")
                    .withColumnRenamed("air_dst","air2_dst")
                    .withColumnRenamed("air_tz","air2_tz")
                   .withColumnRenamed("air_region","air2_region")
                   .withColumnRenamed("air_name","air2_name")
                   )
         

df = df.join(df_airports_bu_1,
                       df_flights_bu.fl_dest == df_airports_bu_1.air2_faa,
                       "left")




In [ ]:
row = df.count()
col = len(df.columns)
print(f'Number of Rows are: {row}')
print(f'Number of Columns are: {col}')


### Pergunta 2

In [ ]:
df2 = df.groupBy('air2_region').agg(F.expr('count(distinct air2_name)').alias('Airports/region')).show()

### Pergunta 3

In [ ]:
df3 = df.agg(F.max(F.col('air_alt')-F.col('air2_alt')).alias("TOP alt difference")).show()


### Pergunta 4

In [ ]:
df4 = df.withColumn('sum_delay', F.col('fl_dep_delay') + F.col('fl_arr_delay'))

#df4 = df4.groupby(['fl_carrier']).agg({'sum_delay':'mean'}).show()

df4.filter((df.fl_dep_delay > 0) | (df.fl_arr_delay > 0) ).agg({"sum_delay" : "avg"}).show()

### Pergunta 5

In [ ]:
df5 = df.groupby(['air2_region']).agg({'fl_arr_delay':'mean','fl_dep_delay':'mean'}).show()

### Pergunta 6

In [ ]:
#df6 = df.groupby(F.year(F.col('fl_dep_datetime'))).agg({'fl_arr_delay':'sum','fl_dep_delay':'sum'}).show()
df6 = df.withColumn('sum_delay', F.col('fl_dep_delay') + F.col('fl_arr_delay'))

df6.filter((df.fl_dep_delay > 0) | (df.fl_arr_delay > 0) ).groupBy(F.year('fl_dep_datetime')).sum('sum_delay').show()

### Pergunta 7

In [ ]:
regs = ['ALASKA','MAINLAND-EAST','MAINLAND-WEST']

df7 = df.filter((df.fl_dep_delay > 0) | (df.fl_arr_delay > 0) ).filter(F.col('air2_region').isin(regs))
df7_origin = (df
                       .filter((df.fl_dep_delay > 0) | (df.fl_arr_delay > 0) )
                       .groupBy(F.col('air2_region'), F.year(F.col('fl_dep_datetime')))
                       .sum('fl_dep_delay'))

df7_dest = (df
                     .filter((df.fl_dep_delay > 0) | (df.fl_arr_delay > 0) )
                     .groupBy(F.col('air2_region'), F.year(F.col('fl_dep_datetime')))
                     .sum('fl_arr_delay'))

df7_origin.show()
df7_dest.show()





### Pergunta 8

In [ ]:
df8 = df.agg({"fl_air_time" : "avg"}).show()


### Pergunta 9

In [ ]:
df9 = df.groupBy('air2_region').avg('fl_air_time').show()

### Pergunta 10

In [ ]:
df10 = df.groupby(['fl_origin','fl_dest']).agg({'fl_air_time':'mean'}).show()

### Pergunta 11

In [ ]:
df.agg({'fl_air_time' : 'sum'}).show()

### Pergunta 12

In [ ]:
df.groupBy('air2_region').sum('fl_air_time').show()

### Pergunta 13

In [ ]:
df.agg({'fl_distance': 'avg'}).show()

### Pergunta 14

In [ ]:
df.groupBy('air2_region').avg('fl_distance').show()

### Pergunta 15

In [ ]:
df.groupBy('air_faa','air2_faa').avg('fl_distance').show()

### Pergunta 16

In [ ]:
df.groupBy(F.year(F.col('fl_dep_datetime'))).sum('fl_distance').show()

### Pergunta 17

In [ ]:
df.groupBy('air2_region').sum('fl_distance').show()

### Pergunta 18

In [ ]:
df_seats_avg = df.groupBy('air_faa','air2_faa').avg('pl_seats')
df_seats_avg = df_seats_avg.withColumn('avg(pl_seats)', F.ceil(F.col('avg(pl_seats)')))
df_seats_avg.show()

### Pergunta 19

In [ ]:
df.groupBy(F.year(F.col('fl_dep_datetime'))).sum('pl_seats').show()

### Pergunta 20

In [ ]:
df.groupBy('air2_faa').count().orderBy(F.col('count').desc()).show()

### Pergunta 21

In [ ]:
df.groupBy('air_faa').count().orderBy(F.col('count').desc()).show()

### Pergunta 22

In [ ]:
(df.select('air_faa','air2_faa','fl_distance')
.filter((F.col('air_faa') == 'PDX')| (F.col('air2_faa') == 'PDX'))
.orderBy(F.col('fl_distance').desc()).show())

### Pergunta 23

In [ ]:
(df.groupBy(F.month(F.col('fl_dep_datetime')), F.col('air2_faa'))
 .count().orderBy(F.col('count').desc()).show())

### Pergunta 24

In [ ]:
df.groupBy('pl_model').count().orderBy(F.col('count').desc()).show()

### Pergunta 25

In [ ]:
(df.filter(F.col('pl_model').isNotNull()).groupBy('pl_model','air2_faa')
 .count().orderBy(F.col('count').desc()).show())

### Pergunta 26

In [ ]:
df_haul_duration = df.groupBy('fl_haul_duration').avg('pl_engines')
df_haul_duration = df_haul_duration.withColumn('avg(pl_engines)',
                                               F.ceil(F.col('avg(pl_engines)')))
df_haul_duration.show()

### Pergunta 27

In [ ]:
df.groupBy('fl_dep_season').count().orderBy(F.col('count').desc()).show()

### Pergunta 28

In [ ]:
df.groupBy('fl_dep_season','air2_faa').count().orderBy(F.col('count').desc()).show()

### Pergunta 29

In [ ]:

df.groupBy('fl_dep_delay_category').count().orderBy(F.col('count').desc()).show()

### Pergunta 30

In [ ]:
(df.filter(F.col('fl_dep_delay_category').isNotNull()).groupBy('air_faa','air2_faa')
 .count().orderBy(F.col('count').desc()).show())